In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

In [ ]:

# Charger les fichiers CSV
olympic_hosts = pd.read_csv('./Datas/olympic_hosts.csv')
olympic_medals = pd.read_csv('./Datas/olympic_medals.csv')

# Fusionner les datasets sur les colonnes correspondantes
combined_data = pd.merge(olympic_medals, olympic_hosts, left_on='slug_game', right_on='game_slug')

# Agréger les données pour obtenir le nombre de médailles par pays pour chaque jeu
medals_by_country = combined_data.groupby(['game_slug', 'game_year', 'country_name', 'medal_type']).size().unstack(fill_value=0)

# Réinitialiser l'index pour faciliter la manipulation
medals_by_country = medals_by_country.reset_index()

# Encoder les noms des pays
label_encoder = LabelEncoder()
medals_by_country['country_code'] = label_encoder.fit_transform(medals_by_country['country_name'])

# Créer un mapping des codes de pays aux noms des pays
country_mapping = dict(zip(medals_by_country['country_code'], medals_by_country['country_name']))

# Trier les données par année pour chaque pays
medals_by_country = medals_by_country.sort_values(by=['country_code', 'game_year'])

# Remplir les valeurs manquantes pour éviter les erreurs de séquence
medals_by_country = medals_by_country.fillna(0)

# Séparer les caractéristiques et la cible
features = medals_by_country[['game_year', 'country_code', 'BRONZE', 'GOLD', 'SILVER']].values
target = medals_by_country[['BRONZE', 'GOLD', 'SILVER']].values

# Vérifier les dimensions des données
print("Features shape:", features.shape)
print("Target shape:", target.shape)


In [ ]:

# Définir la longueur des séquences temporelles
sequence_length = 3

# Utiliser toutes les données disponibles pour l'entraînement
train_features = features
train_target = target

# Créer les générateurs de séquences temporelles pour l'entraînement
train_generator = TimeseriesGenerator(train_features, train_target, length=sequence_length, batch_size=1)

# Définir et compiler le modèle LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, features.shape[1])))
model.add(Dense(3))
model.compile(optimizer='adam', loss='mse')

# Entraîner le modèle
model.fit(train_generator, epochs=20)

In [ ]:

# Préparer les dernières séquences pour chaque pays pour prédire les JO 2024
last_sequences = []
countries = []

for country_code in medals_by_country['country_code'].unique():
    country_data = medals_by_country[medals_by_country['country_code'] == country_code]
    if len(country_data) >= sequence_length:
        last_sequence = country_data[['game_year', 'country_code', 'BRONZE', 'GOLD', 'SILVER']].values[-sequence_length:]
        last_sequences.append(last_sequence)
        countries.append(country_code)

last_sequences = np.array(last_sequences)

# Faire des prédictions pour les JO 2024
jo2024_predictions = []

for seq in last_sequences:
    pred = model.predict(seq[np.newaxis, :, :])
    jo2024_predictions.append(pred[0])

# Organiser les prédictions dans un tableau
predictions_df = pd.DataFrame(jo2024_predictions, columns=['TotalBronze', 'TotalGold', 'TotalSilver'])
predictions_df['Country'] = [country_mapping[code] for code in countries]



In [ ]:
# Calculer le total des médailles
predictions_df['TotalMedals'] = predictions_df['TotalBronze'] + predictions_df['TotalGold'] + predictions_df['TotalSilver']

# Trier par ordre croissant du total des médailles et sélectionner le top 10
top_10_df = predictions_df.sort_values(by='TotalMedals', ascending=False).head(10)

# Réorganiser les colonnes
top_10_df = top_10_df[['Country', 'TotalGold', 'TotalSilver', 'TotalBronze', 'TotalMedals']]

# Exporter les résultats du top 10 en JSON
top_10_df.to_json('top_10_predictions.json', orient='records', lines=True)